# **Functional Programming - Main Patterns**

This is based on the book "Learning Functionl Programming" (Jack Widman).

## 1. Functor Patterns

A functor is like a composable container. The formal definition comes from Category
Theory. A functor $F$ takes a type $T$ to $FT$, and takes a function
$f:T \to T'$ to a function $Ff: FT \to FT'$.
Another description is that a functor is a container together with a `map`
function that allows for composition. A simple example of a functor
is the `Vector` functor. This functor takes any type `T` and turns
into `Vector{T}`. Also, it has a `fmap` function that works
by applying the function `f` to every element of `v::Vector{T}`.
Let's do this example by creating our own functor `List` built from `Vector`.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/FunctionalProgramming`


In [13]:
List{T} = Vector{T}
List(T::Type) = Vector{T}

# List(x...) = [x...]
# Note here that using List for instantianting the vector
# will cause problems if one tries to do List(List(1)).
# Hence, we solve this by calling the function
# `list`.
# Another solution would be to declare
# List(x...) = [x...] and
# List(x::List...) = [x...]

list(x...) = [x...]
fmap(f::Function, a::List{T}) where T = map(f,a)

list(1) isa List{Int}
fmap(x->x^2, list(1,2,3))
List(f::Function) = a::List -> fmap(f,a);

Here is another simple example of a functor. 

In [14]:
struct F{T}
    x::T
end

fmap(f::Function, a::F{T}) where T = F(a.x)

fmap (generic function with 2 methods)

Another useful functor is the `Option` or `Maybe` functor. 
Julia actually has an implementation of this under the `Some` type.

In [15]:
Option{T} = Union{Some{T},Nothing}
@show Some(1) isa Option{Int}
@show something(Some(1)) == 1
Option(::Nothing) = nothing
Option(x) = Some(x)
Option() = nothing

fmap(f, a::Nothing) = nothing
fmap(f, a::Some{T}) where T = Some(f(something(a)))
Option(f::Function) = a::Option -> fmap(f,a)

l = list(Some(1),nothing,Some(8))

Some(1) isa Option{Int} = true
something(Some(1)) == 1 = true


3-element Vector{Union{Nothing, Some{Int64}}}:
 Some(1)
 nothing
 Some(8)

Suppose we want to add 1 to every element in our list. The problem is,
our list is composed of `Option{T}` type elements. So a simple `fmap(f, l)` won't
work.

In [16]:
fmap(x->x+1, l)

LoadError: MethodError: no method matching +(::Some{Int64}, ::Int64)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +([91m::Rational[39m, ::Integer) at rational.jl:313
[0m  ...

The error here is because we are trying to sum 1 to `Some(1)` and similarly,
we are trying to sum 1 to `nothing`.
For this to work, we have to lift our function `x->x+1` to be a function
that acts on `Option{T}`. We know how to do this, simply use the functor
`Option(x->x + 1)`.

In [17]:
fmap(Option(x->x+1),l)

3-element Vector{Union{Nothing, Some{Int64}}}:
 Some(2)
 nothing
 Some(9)

It worked. Note that instead of using `fmap` we could've lifted
`x->x+1` using `List ∘ Option`. 

In [18]:
(List ∘ Option)(x->x+1)(l)

3-element Vector{Union{Nothing, Some{Int64}}}:
 Some(2)
 nothing
 Some(9)

## 2. Monoids

Our second pattern are monoids. Monoids are a triple $(M, e, \otimes)$ where
$M$ is a set, $e$ is a neutral element and $\otimes$ is a binary operator that is
associative. In categorical terms, we can consider that $(M,e \otimes)$ is a category $\mathcal M$,
where $M$ is an object, $e:M\to M$ is an endomorphism (a morphism
from $M$ to itself) and $\otimes$ is the composition operation. Thus, 
$f \circ e = f \otimes e = f = e \otimes f$ for every $f:M \to M$.

In programming, $M$ is a type, $e$ is a value of type $M$ and $\otimes$
is a binary function $\otimes(a::M,b::M)::M$. So this collection defines a monoid
*if* $a \otimes e = a = e \otimes a$ for every $a \in M$, and
$a \otimes(b \otimes c) = (a \otimes b) \otimes c$.

Our first example of a monoid will be (`String`,`""`,`*`). 

In [19]:
neutral(::Type{String}) = ""
⊗(s1::String, s2::String) = s1 * s2

"ok" ⊗ "nn" ⊗ "mm" ⊗ neutral(String)

"oknnmm"

Our second example is (`Real`,`0`,`+`) 

In [20]:
neutral(::Type{Real}) = 0
⊗(n1::Real, n2::Real) = n1 + n2

neutral(Real) ⊗ 1 ⊗ 2 ⊗ neutral(Real)

3

From two monoids, we can create a new monoid by taking the "tuple" (product), i.e.
$(M \times M', e_M \times e_{M'}, \otimes_M \times \otimes_{M'})$.

In [21]:
neutral(::Type{Tuple{M1, M2}}) where {M1,M2} = (neutral(M1),neutral(M2))
⊗(x1::Tuple{M1,M2}, x2::Tuple{M1,M2}) where {M1,M2} = (x1[1] ⊗ x2[1], x1[2] ⊗ x2[2])

⊗ (generic function with 3 methods)

In [22]:
("one",1) ⊗ ("two",2) ⊗ neutral(Tuple{String, Real})

("onetwo", 3)

### Monoids as Traits

Note that if we want to know if an specific type `T` has a "monoid interface", i.e.
the `neutral(T::Type)` and `⊗(x::T, y::T)`, is defined, then we need to
"manually" check.

In [23]:
abstract type Monoid end
struct IsMonoid <: Monoid end
struct NotMonoid <: Monoid end


Base.:&(m1::IsMonoid,m2::IsMonoid) = IsMonoid()
Base.:&(m1::NotMonoid,m2::IsMonoid) = NotMonoid()
Base.:&(m1::IsMonoid,m2::NotMonoid) = NotMonoid()
Base.:&(m1::NotMonoid,m2::NotMonoid) = NotMonoid()

Monoid(::Type) = NotMonoid()
Monoid(::Type{<:String}) = IsMonoid()
Monoid(::Type{<:Real}) = IsMonoid()
Monoid(::Type{Tuple{T1,T2}}) where {T1,T2} = Monoid(T1) & Monoid(T2);

In [24]:
@show Monoid(Tuple{String,Real})
@show Monoid(String);
@show Monoid(Bool);

Monoid(Tuple{String, Real}) = IsMonoid()
Monoid(String) = IsMonoid()
Monoid(Bool) = IsMonoid()


In [25]:
ismonoid(x::T) where T  = ismonoid(Monoid(T))
ismonoid(::IsMonoid) = true
ismonoid(::NotMonoid) = false

ismonoid (generic function with 3 methods)

In [26]:
@show ismonoid(1.0)
@show ismonoid("example");

ismonoid(1.0) = true
ismonoid("example") = true


## 3. Monads

A monad is a monoid in the category of endofunctors, i.e.
for an endofunctor $M$, we define a monad as
$(M, \eta, \mu)$, where $\eta$ and $\mu$ are natural transformations. We call
$\eta$ the unit, and $\mu$ is the join. Another equivalent way of defining a monad
is with the endofunctor $M$ plus the unit and a bind operator. The bind is derived
from the join, and vice-versa. The notebook on Category Theory section explains it deeper.


A functor is this construction that has a `fmap` function associated, and a monoid
has a neutral element and an associate composition operation. Thus, consider
a functor such as `F{T}` in o

In [27]:
abstract type Monad end
struct IsMonad <: Monoid end
struct NotMonad <: Monoid end


Base.:&(m1::IsMonoid,m2::IsMonoid) = IsMonoid()
Base.:&(m1::NotMonoid,m2::IsMonoid) = NotMonoid()
Base.:&(m1::IsMonoid,m2::NotMonoid) = NotMonoid()
Base.:&(m1::NotMonoid,m2::NotMonoid) = NotMonoid()

Monoid(::Type) = NotMonoid()
Monoid(::Type{<:String}) = IsMonoid()
Monoid(::Type{<:Real}) = IsMonoid()
Monoid(::Type{Tuple{T1,T2}}) where {T1,T2} = Monoid(T1) & Monoid(T2);

A simpler description of a monad is that is a functor has a `flatmap` and a `unit`.
Remember that every functor already has a `fmap`. Yet, with a `flatmap` and a
`unit`, one can recover `fmap`.

In [28]:
struct unit{M} end

In [29]:
unit{Option}(x) = Option(x)
unit{List}(x...)   = list(x...)

In [30]:
getoption(x::Some) = something(x)
getoption(x::Nothing) = nothing
join(x::Option{<:Option{T}}) where T = getoption(x)
join(x::List{<:List{T}}) where T = vcat(x...)

# also called bind
flatmap(f, x::Option) = join(fmap(f, x))
flatmap(f, x::List)   = join(fmap(f, x))

flatmap (generic function with 2 methods)

In [31]:
# fmap(join,list(Option(1),Option(),Option(2)))

Let's do an example. Consider our functor `Option{T}`.
First, the `unit` function takes an object of type `T` and turns it
into `Option{T}`.

In [32]:
println(Option(x::String -> x=="" ? nothing : Some(0))(unit{Option}("abc")))
println(fmap(x::String -> x=="" ? nothing : Some(0), unit{Option}("abc")));
println(join(fmap(x::String -> x=="" ? nothing : Some(0), unit{Option}("abc"))));

Some(Some(0))
Some(Some(0))
Some(0)


In [33]:
println(flatmap(x::String -> x=="" ? nothing : Some(0), unit{Option}("abc")));

Some(0)


## INCOMPLETE EXAMPLE

To illustrate how this can be useful, let's again consider the example with 
the user database.

In [83]:
struct User
    name::String
end
DATABASE = Dict( 4 =>User("James"), 2 =>User("John"), 1 =>User("Peter"));

function getuserfromdb(uid::Int)::Option
    result = nothing
    if uid in keys(DATABASE)
        result = DATABASE[uid]
    end
    return Option(result)
end
getuserfromdb(1)

Some(User("Peter"))

We have a list of user ids and we want to apply the `getuserfromdb`
function.

In [95]:
userids = list(1,2,3)
fmap(getuserfromdb,userids)

3-element Vector{Union{Nothing, Some{User}}}:
 Some(User("Peter"))
 Some(User("John"))
 nothing

It worked, but we got a list of `Options`, and we actually wanted a list of users. 
Let's try with the `flatmap`.

In [85]:
# flatmap(getuserfromdb, userids)
flatmap(getuserfromdb,userids)
# flatmap(x->list(x),List(1,2,3))

3-element Vector{Union{Nothing, Some{User}}}:
 Some(User("Peter"))
 Some(User("John"))
 nothing

In [ ]:
# flatmap(w::Writer{Writer{T,M},M}) where T where M = Writer(w.a.a,w.log ⊗ w.a.log )

In [ ]:
f(x) = x==1 ? nothing : Some(0)

flatmap(f,List(1,2,3))

In [ ]:
flatmap(f,Some("ok"))